In [1]:
# imports
import numpy as np
import os 
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import xarray as xr
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import dash_bootstrap_components as dbc

In [2]:
# set directory names 
indir_1dbl = "output"
indir_sensitivity_data = os.path.join("Outputs","Ratio_constant")

In [3]:
# set column names for 1dbl data 
columns = ['zn', 'z', 'p', 'u', 'v', 'th', 'qv', 'ql', 'rh','uw', 'vw', 'wth', 'k_m', 'k_h', 'c_vis']

# initialse data frame 
alldata = pd.DataFrame()

# loop over desired times 
for time in np.arange(start=1200, stop=52800, step=1200):

    # read file 
    with open(os.path.join(indir_1dbl,"arm_shcu_t_"+str(time)+".dat"), 'r') as f:
        lines = f.readlines()

    # initialise temporary df to store data for current time 
    data = pd.DataFrame()

    # loop over lines 
    for ii, line in enumerate(lines):

        # ignore the first line 
        if ii == 0:
            continue 

        # split the data based on tab spaces 
        newline = line.split()

        # remove any lines that aren't the expected length 
        if len(newline) != 15:
            continue
        else: 

            # convert data to pandas Series and concatenate to data frame 
            newline = pd.Series(newline).astype(float)
            data = pd.concat([data, newline], axis=1)

    # trnaspose the data 
    data = data.T

    # set column names 
    data.columns = columns

    # set time column 
    data["time"] = np.repeat(time, len(data))

    # put time column to front of the data frame 
    data = data.iloc[:, [-1, 0, 1,2,3,4,5,6,7,8,9,10,11,12,13,14]]

    # append current data frame  to the bottom of the 'main' data frame 
    alldata = pd.concat([alldata, data], axis=0)


In [4]:
# set column names for 1dbl data 
columns = ['zn', 'z', 'p', 'u', 'v', 'th', 'qv', 'ql', 'rh','uw', 'vw', 'wth', 'k_m', 'k_h', 'c_vis']

def make_stdf(number):
    # initialse data frame 
    stdf = pd.DataFrame()

    #set path name for current run 
    curpath = os.path.join(indir_sensitivity_data, 'output_'+number)

    # loop over desired times 
    for time in np.arange(start=1200, stop=52800, step=1200):

        # read file 
        with open(os.path.join(curpath,"arm_shcu_t_"+str(time)+".dat"), 'r') as f:
            lines = f.readlines()

        # initialise temporary df to store data for current time 
        data = pd.DataFrame()

        # loop over lines 
        for ii, line in enumerate(lines):

            # ignore the first line 
            if ii == 0:
                continue 

            # split the data based on tab spaces 
            newline = line.split()

            # remove any lines that aren't the expected length 
            if len(newline) != 15:
                continue
            else: 

                # convert data to pandas Series and concatenate to data frame 
                newline = pd.Series(newline).astype(float)
                data = pd.concat([data, newline], axis=1)

        # trnaspose the data 
        data = data.T

        # set column names 
        data.columns = columns

        # set time column 
        data["time"] = np.repeat(time, len(data))

        # put time column to front of the data frame 
        data = data.iloc[:, [-1, 0, 1,2,3,4,5,6,7,8,9,10,11,12,13,14]]

        # append current data frame  to the bottom of the 'main' data frame 
        stdf = pd.concat([stdf, data], axis=0)

    return stdf

In [5]:
stdf_01 = make_stdf("01")
stdf_02 = make_stdf("02")
stdf_03 = make_stdf("03")
stdf_04 = make_stdf("04")
stdf_05 = make_stdf("05")
stdf_06 = make_stdf("06")
stdf_07 = make_stdf("07")

In [6]:
## function to take time in seconds and convert to __h __ (min) 
def convert_to_hr_min(value):
     return str(int(np.floor(value/3600)))+"h"+str(int(60*((value/3600)-np.floor(value/(3600)))))


In [7]:
# initialise app 
app = Dash(__name__)

# set app layout =======================================================================================
app.layout = html.Div([

    # titles 
    html.H1("Boundary Layer analysis"),
    html.H2("Single profile"),
    html.Label("Select variable:"),

    # dropdown menu 
    dcc.Dropdown(
        options = stdf_01.columns[3:],
        value = "th",
        id = 'variable-dropdown',
        className = 'dropdown',
        style=dict(width='40%')
    ),
    html.Br(),

    # time slider
    html.Label('Select time:'),
    dcc.Slider(1200, 51600, 2400,
        value = 1200,
        id = 'time-slider',
        marks = {str(i):convert_to_hr_min(i) for i in np.arange(start=1200, step=2400, stop=51700)},
        className = 'slider' ),
    html.Br(),

    # graph 
    dcc.Graph(id='vertical-profile', figure=go.Figure())
],
style = {'padding':'30px'})

# callbacks ===============================================================================================
@app.callback(
    Output('vertical-profile', 'figure'),
    Input('time-slider', 'value'),
    Input('variable-dropdown', 'value')
)
def update_plot(selected_time, plot_var_1dbl):

    # change z coordinate for some vars in 1dbl data
    if plot_var_1dbl in ["u", "v"]:
        z_type = "zn"
    else:
        z_type ="z"

    data_filtered = alldata[alldata["zn"]<=3000]
    minx = min(data_filtered[plot_var_1dbl])
    maxx = max(data_filtered[plot_var_1dbl])
    data_filtered = data_filtered[data_filtered["time"]==float(selected_time)]
    
    # filter the other plotting data 
    stdf01_filtered = stdf_01[stdf_01["zn"]<=3000]
    stdf01minx = min(stdf01_filtered[plot_var_1dbl])
    stdf01maxx = max(stdf01_filtered[plot_var_1dbl])
    stdf01_filtered = stdf01_filtered[stdf01_filtered["time"]==float(selected_time)]

    stdf02_filtered = stdf_02[stdf_02["zn"]<=3000]
    stdf02minx = min(stdf02_filtered[plot_var_1dbl])
    stdf02maxx = max(stdf02_filtered[plot_var_1dbl])
    stdf02_filtered = stdf02_filtered[stdf02_filtered["time"]==float(selected_time)]

    stdf03_filtered = stdf_03[stdf_03["zn"]<=3000]
    stdf03minx = min(stdf03_filtered[plot_var_1dbl])
    stdf03maxx = max(stdf03_filtered[plot_var_1dbl])
    stdf03_filtered = stdf03_filtered[stdf03_filtered["time"]==float(selected_time)]

    stdf04_filtered = stdf_04[stdf_04["zn"]<=3000]
    stdf04minx = min(stdf04_filtered[plot_var_1dbl])
    stdf04maxx = max(stdf04_filtered[plot_var_1dbl])
    stdf04_filtered = stdf04_filtered[stdf04_filtered["time"]==float(selected_time)]

    stdf05_filtered = stdf_05[stdf_05["zn"]<=3000]
    stdf05minx = min(stdf05_filtered[plot_var_1dbl])
    stdf05maxx = max(stdf05_filtered[plot_var_1dbl])
    stdf05_filtered = stdf05_filtered[stdf05_filtered["time"]==float(selected_time)]

    stdf06_filtered = stdf_06[stdf_06["zn"]<=3000]
    stdf06minx = min(stdf06_filtered[plot_var_1dbl])
    stdf06maxx = max(stdf06_filtered[plot_var_1dbl])
    stdf06_filtered = stdf06_filtered[stdf06_filtered["time"]==float(selected_time)]
    
    stdf07_filtered = stdf_07[stdf_07["zn"]<=3000]
    stdf07minx = min(stdf07_filtered[plot_var_1dbl])
    stdf07maxx = max(stdf07_filtered[plot_var_1dbl])
    stdf07_filtered = stdf07_filtered[stdf07_filtered["time"]==float(selected_time)]

    # set limits 
    totalminx = min([stdf01minx, stdf02minx, stdf03minx, stdf04minx, stdf05minx, stdf06minx, stdf07minx, minx])
    totalmaxx = max([stdf01maxx, stdf02maxx, stdf03maxx, stdf04maxx, stdf05maxx, stdf06maxx, stdf07maxx, maxx])

    # initialise figure 
    fig1 = go.Figure()

    # # plot 1dbl data 
    # fig1.add_trace(go.Scatter(x=np.array(data_filtered[plot_var_1dbl]),
    #                 y=np.array(data_filtered[z_type]),
    #                 mode="lines",
    #                 name = '1dbl',
    #                 line = dict(width=4),
    #                 opacity = 0.5)),

    # plot les data 
    fig1.add_trace(go.Scatter(x=np.array(stdf01_filtered[plot_var_1dbl]),
                    y=np.array(stdf01_filtered[z_type]),
                    mode="lines",
                    name = '60%',
                    line = dict(width=4),
                    opacity =0.5)),

    fig1.add_trace(go.Scatter(x=np.array(stdf02_filtered[plot_var_1dbl]),
                    y=np.array(stdf02_filtered[z_type]),
                    mode="lines",
                    name = '80%',
                    line = dict(width=4),
                    opacity =0.5)),

    fig1.add_trace(go.Scatter(x=np.array(stdf03_filtered[plot_var_1dbl]),
                y=np.array(stdf03_filtered[z_type]),
                mode="lines",
                name = '90%',
                line = dict(width=4),
                opacity =0.5)),

    fig1.add_trace(go.Scatter(x=np.array(stdf04_filtered[plot_var_1dbl]),
                y=np.array(stdf04_filtered[z_type]),
                mode="lines",
                name = 'control',
                line = dict(width=4),
                opacity =0.5)),

    fig1.add_trace(go.Scatter(x=np.array(stdf05_filtered[plot_var_1dbl]),
                y=np.array(stdf05_filtered[z_type]),
                mode="lines",
                name = '110%',
                line = dict(width=4),
                opacity =0.5)),

    fig1.add_trace(go.Scatter(x=np.array(stdf06_filtered[plot_var_1dbl]),
                y=np.array(stdf06_filtered[z_type]),
                mode="lines",
                name = '120%',
                line = dict(width=4),
                opacity =0.5)),

    fig1.add_trace(go.Scatter(x=np.array(stdf07_filtered[plot_var_1dbl]),
                y=np.array(stdf07_filtered[z_type]),
                mode="lines",
                name = '140%',
                line = dict(width=4),
                opacity =0.5)),

    # set title, aspect ratio and limits 
    fig1.update_layout(title=f'Time = {selected_time}, {convert_to_hr_min(selected_time)}', width =500, height=600)
    fig1.update_layout(xaxis_title=f'{plot_var_1dbl}')
    fig1.update_layout(yaxis_title=f'Height [m]')
    fig1.update_xaxes(range=[totalminx,totalmaxx])
    fig1.update_yaxes(range=[0,3000])

    return fig1

# run app 
if __name__ == '__main__':
    host = "127.0.0.1"
    port = 8054  # change this if needed
    print(f"\n🚀 Starting Dash app at: http://{host}:{port}/\n")
    app.run(debug=True, host=host, port=port)


🚀 Starting Dash app at: http://127.0.0.1:8054/

